In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset, load_dataset

In [22]:
thread = open("./verbalist/datasets/bugurt_thread/messages30.html").read()

soup = BeautifulSoup(thread)
all_messages = soup.find_all("div", class_="message")

In [23]:
len(all_messages)

1123

In [24]:
import markdownify

text = str(all_messages[100].find("div", class_="text"))
# text
text = str(
    markdownify.markdownify(
        text,
    )
)
text = text.replace("предложено через", "")
text = text.replace("[@tgbtbot](https://t.me/tgbtbot)", "")
text = text.replace("[БУГУРТ-ТРЕД](https://t.me/bugurtthread)", "")
print(text)


МОРАЛЬ СЕЙ БАСНИ ТАКОВА  
@  
ТОЛПОЮ ГАСЯТ ДАЖЕ ЛЬВА  
@  
НО НЕ В ЭТОМ СУТЬ ПОВЕРЬТЕ  
@  
ТОЛПОЮ ГАСЯТ ТОЛЬКО ЧЕРТИ
 


In [25]:
text.count("@")

3

In [26]:
import re

dataset = []

for num in range(30, 45):
    thread = open(f"./verbalist/datasets/bugurt_thread/messages{num}.html").read()

    soup = BeautifulSoup(thread)
    all_messages = soup.find_all("div", class_="message")
    for i in range(len(all_messages)):
        text = str(all_messages[i].find("div", class_="text"))
        # text
        text = str(
            markdownify.markdownify(
                text,
            )
        )
        text = text.replace("предложено через", "")
        text = text.replace("предложено  через", "")
        text = text.replace("[@tgbtbot](https://t.me/tgbtbot)", "")
        text = text.replace("[БУГУРТ-ТРЕД](https://t.me/bugurtthread)", "")
        text = re.sub(r"\[(.*?)\]\(.*?\)", "", text).strip()
        if "@" in text:
            if text.count("@") >= 3:
                # print(text)
                dataset.append({"bugurt": text})
                # print("=" * 100)

In [28]:
len(dataset)

8360

In [29]:
new_dataset = Dataset.from_list(dataset)
new_dataset = new_dataset.train_test_split(test_size=5000, seed=42)
new_dataset = new_dataset['test']
new_dataset.push_to_hub("dim/raw_bugurts_5k")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.86it/s]
Updating downloaded metadata with the new split.


In [44]:
dataset = Dataset.from_list(dataset)

In [45]:
dataset.to_csv("./verbalist/datasets/bugurt_thread/bugurt_thread.csv")

Creating CSV from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 48.52ba/s]


2831798

### sumarize bugurts


In [28]:
import openai
import pandas as pd
import numpy as np
import time

openai.api_key = open("./chat_gpt_token").read()


def chat_with_chatgpt(prompt, model="gpt-3.5-turbo"):
    chat_completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    return chat_completion["choices"][0]["message"]["content"].strip()


user_prompt = "Hello world."
chatbot_response = chat_with_chatgpt(user_prompt)
print(chatbot_response)

Hello! How can I assist you today?


In [43]:
text = dataset[10]["bugurt"]

prompt = f"""
Текст: {text}

Напиши бугурт на основе текста по типу:
Напиши бугурт на тему...
или
Напиши бугурт про то как...
или
Напиши бугурт о том как...

Бугурт должен быть довольно коротким 3-6 слов.
"""
print(text)
chatbot_response = chat_with_chatgpt(prompt, model="gpt-4")
print(chatbot_response)

"у меня своя рок группа"  
@  
ЕДИНСТВЕННЫМИ ПАЛОЧКАМИ В ЖИЗНИ БАРАБАНЩИКА БЫЛИ ПАЛОЧКИ ОТ СУШИ  
@  
БАСИСТ ДАЖЕ НЕ ЗНАЕТ ЧТО ОН В ГРУППЕ  
@  
ВОКАЛИСТА НЕТ  
@  
ВСЕ ПЕСНИ ЭТО ГАЧИКАВЕРЫ НА ЕГОРА ЛЕТОВА
Напиши бугурт на тему "безвокальная рок группа":
"Молчание голоса, звук рока."


### export labeled bugurts

In [13]:
import pandas as pd
from datasets import Dataset

dataset = pd.read_csv("./verbalist/datasets/bugurt_thread/bugurt_thread_labeled_v2.csv")
dataset = dataset[~dataset['prompt'].isna()]
dataset = dataset.to_dict('records')



In [14]:
for i in range(len(dataset)):
    dataset[i]['bugurt'] = dataset[i]['bugurt'].lower() 

In [15]:
dataset[4]

{'bugurt': '2015 год  \n@  \nты активно учавствуешь в срачах в комментариях, угараешь с рандомами над пастами про говно, клеймишь админа петухом за форс мемов "ня, пока" и "кека"  \n@  \nс основного аккаунта  \n@  \n2022 год  \n@  \nрешаешь оставить шуточный комментарий в %пабликнейм%  \n@  \n"славка сычев комментирует запись"  \n@  \n"ваш друг славка сычев сидит в %пабликнейм%, возможно вам понравится это сообщество"  \n@  \n"вы знакомы со славкой сычевым? у вас 1 общий друг и 1 общее сообщество, в котором он жестко наваливает кринжа в комментариях"  \n@  \nмодераторы вк лично отправляют ссылку на твой комментарий всем друзьям и родственникам  \n@  \nскриншот комментария рассылается твоим коллегам в офис через факс  \n@  \nтекст с твоим комментарием звучит в рекламе перед каждой попыткой послушать музыку  \n@  \nскриншот комментария развешивается на биллбордах над мкадом  \n@  \nк тебе в хату врывается батя на "серьезный разговор" о том, что они с матерью тебя не таким воспитывали',
 

In [16]:
dataset = Dataset.from_list(dataset)
dataset.push_to_hub("dim/bugurt_thread_prompts")

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  3.92it/s]
Updating downloaded metadata with the new split.


In [17]:
dataset

Dataset({
    features: ['bugurt', 'prompt'],
    num_rows: 223
})

### create bugurt completion

In [36]:
dataset = load_dataset("dim/raw_bugurts_5k")
dataset = dataset['train']
dataset = dataset.to_list()
dataset[0]

Found cached dataset parquet (/home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--raw_bugurts_5k-bcb10a500f02031d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00, 971.80it/s]


{'bugurt': 'ТЫ ПАРЕНЬ 16 ЛВЛ   \n@  \nВ ВЕКА СТОИТ ФАМИЛИЯ МИРНЫЙ/ВОЛКОВ/ЗАБИВНОЙ/ВЫСОЦКИЙ/\\*город нейм\\* СКИЙ/НУРМИНСКИЙ/ЕТС  \n@  \nВ ОПИСАНИИ ПРОФИЛЯ НАПИСАНА ЛИБО ПАЦАНСКАЯ ЦИТАТА АЛЯ "ВОЛК НЕ ТОТ КТО ВОЛК, А ТОТ КТО ВОЛК" ЛИБО ДАТА СБРОСА ЛИСТВЫ С НАСТЕЙ МИРНОЙ, ЧТОБВ ВСЕ ЗНАЛИ, ЧТО ТЫ НЕ ТАКОЙ СЫЧ КАК ОСТАЛЬНЫЕ  \n@  \nНА АВАТАРКЕ СТОИТ ФОТО ВОЛКА/ФОТО ИЗ ИНТЕРНЕТА, ГДЕ ЛИЦО ЗАКРЫТО ЭМОДЗИ ЧЕРТА/ФОТО В ШАПКЕ С ЛИНЗАМИ НА ФОНЕ ПОЛИЦЕЙСКОГО АВТО  \n@  \nЗАПИСИ НА ТВОЕЙ СТРАНИЧКЕ ПРЕДСТАВЛЯЮТ ИЗ СЕБЯ ЦИТАТЫ С ПРИКРЕПЛЕННОЙ ПЕСНЕЙ НУРМИНСКОГО ИЛИ РЕМИКСОМ КАКОЙ НИБУДЬ ПОПУЛЯРНОЙ ПЕСНИ (ОБЯЗАТЕЛЬНО С РОЗОЙ В НАЗВАНИИ), ЛИБО ФОТОГРАФИЕЙ ТАКОЙ ЖЕ КАК ТВОЯ АВАТАРКА С ТОЙ ЖЕ ПРИКРЕПЛЕННОЙ ПЕСНЕЙ  \n@  \nКАЖДОГО, КТО ПИШЕТ ГАДОСТИ ТЕБЕ/НАСТЕ МИРНОЙ ВЫЗЫВАЕШЬ НА ЗАБИВ И УГРОЖАЕШЬ НАЙТИ  \n@  \nТВОЙ КРУГ ОБЩЕНИЯ СОСТОИТ ТОЛЬКО ИЗ ТЕБЕ ПОДРОБНЫХ   \n@  \nВ РЕАЛЬНОСТИ ТЫ ГНОМОПОДОБНАЯ ТУМБОЧКА ВАНЬКА ТАРАСОВ 160 СМ ВЕСОМ 100 КГ   \n@  \nВСЕРЬЕЗ СЧИТАЕШЬ, ЧТО ЭТО НЕ ТЫ ПОСТЫДНОЕ ЧМО, А ПАРНИ 

In [37]:
new_dataset = []
for i in range(len(dataset)):
    bugurt = dataset[i]['bugurt'].lower().strip() 
    first_line = bugurt.split("@")[0]
    prompt = f"продолжи бугурт: {first_line}".strip()
    new_dataset.append({
		"prompt": prompt,
		"bugurt": bugurt
	})

In [38]:
new_dataset[0]

{'prompt': 'продолжи бугурт: ты парень 16 лвл',
 'bugurt': 'ты парень 16 лвл   \n@  \nв века стоит фамилия мирный/волков/забивной/высоцкий/\\*город нейм\\* ский/нурминский/етс  \n@  \nв описании профиля написана либо пацанская цитата аля "волк не тот кто волк, а тот кто волк" либо дата сброса листвы с настей мирной, чтобв все знали, что ты не такой сыч как остальные  \n@  \nна аватарке стоит фото волка/фото из интернета, где лицо закрыто эмодзи черта/фото в шапке с линзами на фоне полицейского авто  \n@  \nзаписи на твоей страничке представляют из себя цитаты с прикрепленной песней нурминского или ремиксом какой нибудь популярной песни (обязательно с розой в названии), либо фотографией такой же как твоя аватарка с той же прикрепленной песней  \n@  \nкаждого, кто пишет гадости тебе/насте мирной вызываешь на забив и угрожаешь найти  \n@  \nтвой круг общения состоит только из тебе подробных   \n@  \nв реальности ты гномоподобная тумбочка ванька тарасов 160 см весом 100 кг   \n@  \nвсерьез

In [39]:
new_dataset = Dataset.from_list(new_dataset)
new_dataset.push_to_hub("dim/bugurt_completion_prompts")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


In [1]:
from datasets import load_dataset

dataset = load_dataset("dim/bugurt_completion_prompts")
dataset = dataset["train"]
dataset[0]

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 355.15it/s]


Dataset parquet downloaded and prepared to /home/kosenko/.cache/huggingface/datasets/dim___parquet/dim--bugurt_completion_prompts-3dd6cb88dc70b252/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 716.36it/s]


{'prompt': 'продолжи бугурт: ты парень 16 лвл',
 'bugurt': 'ты парень 16 лвл   \n@  \nв века стоит фамилия мирный/волков/забивной/высоцкий/\\*город нейм\\* ский/нурминский/етс  \n@  \nв описании профиля написана либо пацанская цитата аля "волк не тот кто волк, а тот кто волк" либо дата сброса листвы с настей мирной, чтобв все знали, что ты не такой сыч как остальные  \n@  \nна аватарке стоит фото волка/фото из интернета, где лицо закрыто эмодзи черта/фото в шапке с линзами на фоне полицейского авто  \n@  \nзаписи на твоей страничке представляют из себя цитаты с прикрепленной песней нурминского или ремиксом какой нибудь популярной песни (обязательно с розой в названии), либо фотографией такой же как твоя аватарка с той же прикрепленной песней  \n@  \nкаждого, кто пишет гадости тебе/насте мирной вызываешь на забив и угрожаешь найти  \n@  \nтвой круг общения состоит только из тебе подробных   \n@  \nв реальности ты гномоподобная тумбочка ванька тарасов 160 см весом 100 кг   \n@  \nвсерьез